In [1]:
from sim_utils import plot_utils, run_sim
from graph_create import watts_strogatz
from plotly.offline import init_notebook_mode

init_notebook_mode(connected=True)

In [2]:
# Constants
num_vertices = 100
regularity = 4

In [3]:
beta = 0.1
def create_ws(n, k, p, r):
    global beta
    return watts_strogatz(n, k, beta, p, p, r)

## Evenly distributed resources

In [ ]:
run_sim(num_vertices, regularity, create_ws)

## Powerlaw distributed resources

In [ ]:
run_sim(num_vertices, regularity, create_ws, True)